In [1]:
# !pip install -r ../requirements.txt
# !pip install bootstrapped

## Importing Libraries

In [2]:
#Importing libraries
import numpy as np
import pandas as pd
import os
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

In [3]:
# Note: Data Cleaning was performed using Python. The code used for cleaning has been removed for the sake of simplicity.

### 1. Time Series Classification Part 1: Feature Creation/Extraction

#### An interesting task in machine learning is classification of time series. In this problem, we will classify the activities of humans based on time series obtained by a Wireless Sensor Network.

(a) Download the AReM data from:
https://archive.ics.uci.edu/ml/datasets/Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\%29
 The dataset contains 7 folders that represent seven types of activities. In each folder, there are multiple files each of which represents an instant of a human performing an activity. 1 Each file containis 6 time series collected from activities of the same person, which are called avg rss12, var rss12, avg rss13, var rss13, vg rss23, and ar rss23. There are 88 instances in the dataset, each of which contains 6 time series and each time series has 480 consecutive values.

(b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data.

(c) Feature Extraction Classification of time series usually needs extracting features from them. In this problem, we focus on time-domain features. 

    i. Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).
    
#### Answer: Minimum, Maximum, Mean, Median, Standard deviation, Skewness, Kurtosis, k-quantiles etc.

    ii. Extract the time-domain features minimum, maximum, mean, median, stan- dard deviation, first quartile, and third quartile for all of the 6 time series in each instance. You are free to normalize/standardize features or use them directly. 
#### Answer: Check the code cell below.

In [4]:
test = pd.DataFrame()
train = pd.DataFrame()
for root, dirs, files in os.walk("..\data", topdown=False):
    
    for name in files:
        if name in ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']:
            continue
        
        directory = root.split(os.path.sep)[-1]
        
        df = pd.read_csv(root+"/"+name,skiprows=5,header=None,\
                         names=["time","1","2","3","4","5","6"])
        df.drop("time",axis=1,inplace=True)
        app = df.describe().T[['mean','std','min','25%','50%','75%','max']]
        app.columns = ['mean','std','min','Q1','median','Q3','max']
        v = app.unstack().to_frame().sort_index(level=1).T
        v.columns = v.columns.map('_'.join)
        v['class'] = directory
        if directory == "bending1" or directory == "bending2":
            if name == "dataset1.csv" or name == "dataset2.csv":
                test = pd.concat([test,v],axis=0)    
            else:
                train = pd.concat([train,v],axis=0)
        else:
            if name == "dataset1.csv" or name == "dataset2.csv" or name == "dataset3.csv":
                test = pd.concat([test,v],axis=0)
            else:
                train = pd.concat([train,v],axis=0)
                
train = train.reset_index().drop('index',axis=1)
test = test.reset_index().drop('index',axis=1)

In [5]:
#Columns
# train.columns

In [6]:
train.head(5)

,Q1_1,Q3_1,max_1,mean_1,median_1,min_1,std_1,Q1_2,Q3_2,max_2,...,min_5,std_5,Q1_6,Q3_6,max_6,mean_6,median_6,min_6,std_6,class
0,43.00,45.00,47.40,43.954500,44.33,35.00,1.558835,0.00,0.50,1.70,...,29.00,1.999604,0.00,0.94,1.79,0.493292,0.43,0.0,0.513506,bending1
1,39.15,45.00,47.75,42.179813,43.50,33.00,3.670666,0.00,1.12,3.00,...,20.00,3.849448,0.00,1.00,2.18,0.613521,0.50,0.0,0.524317,bending1
2,41.33,42.75,45.75,41.678063,41.75,33.00,2.243490,0.43,0.71,2.83,...,23.67,2.411026,0.00,0.50,1.79,0.383292,0.43,0.0,0.389164,bending1
3,42.50,45.00,48.00,43.454958,43.25,37.00,1.386098,0.00,0.50,1.58,...,8.00,2.488862,0.43,0.87,5.26,0.679646,0.50,0.0,0.622534,bending1
4,43.31,44.67,48.00,43.969125,44.50,36.25,1.618364,0.43,0.50,1.50,...,11.33,3.318301,0.00,0.83,2.96,0.555313,0.49,0.0,0.487826,bending1


In [7]:
test.head(5)

,Q1_1,Q3_1,max_1,mean_1,median_1,min_1,std_1,Q1_2,Q3_2,max_2,...,min_5,std_5,Q1_6,Q3_6,max_6,mean_6,median_6,min_6,std_6,class
0,39.2500,42.00,45.00,40.624792,40.50,37.25,1.476967,0.0,0.50,1.30,...,27.25,2.188449,0.00,1.30,1.92,0.570583,0.43,0.0,0.582915,bending1
1,42.0000,43.67,45.67,42.812812,42.50,38.00,1.435550,0.0,0.50,1.22,...,27.67,1.995255,0.00,1.30,3.11,0.571083,0.43,0.0,0.601010,bending1
2,23.1875,26.50,51.00,24.562958,24.25,12.75,3.737514,0.0,0.71,6.87,...,10.67,3.693786,0.43,0.87,4.97,0.700188,0.50,0.0,0.693720,bending2
3,25.5000,30.00,42.75,27.464604,28.00,0.00,3.583582,0.0,0.50,7.76,...,0.00,5.053642,0.47,1.30,6.76,1.122125,0.83,0.0,1.012342,bending2
4,34.5000,40.25,45.00,37.177042,36.25,24.25,3.581301,1.3,3.13,8.58,...,7.00,2.890347,1.50,3.90,9.34,2.921729,2.50,0.0,1.852600,cycling


In [8]:
print("Test Shape",test.shape)
print("Train Shape",train.shape)

Test Shape (19, 43)
Train Shape (69, 43)


iii. Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature.

In [9]:
df = pd.concat([test,train],ignore_index=True,axis=0)
df_features = df.drop(['class'],axis=1)
df_features.std()

Q1_1        6.153590
Q3_1        5.145600
max_1       4.394362
mean_1      5.336157
median_1    5.440054
min_1       9.569975
std_1       1.772415
Q1_2        0.946386
Q3_2        2.125266
max_2       5.062729
mean_2      1.574181
median_2    1.412244
min_2       0.000000
std_2       0.884102
Q1_3        4.220658
Q3_3        4.171628
max_3       4.875137
mean_3      4.007220
median_3    4.034331
min_3       2.956462
std_3       0.946872
Q1_4        0.843620
Q3_4        1.552504
max_4       2.183625
mean_4      1.166071
median_4    1.145586
min_4       0.000000
std_4       0.458234
Q1_5        6.096465
Q3_5        5.531720
max_5       5.741238
mean_5      5.675589
median_5    5.813782
min_5       6.124001
std_5       1.024767
Q1_6        0.758584
Q3_6        1.523599
max_6       2.518921
mean_6      1.154796
median_6    1.086474
min_6       0.045838
std_6       0.517610
dtype: float64

In [10]:
# df_features.std(ddof=0).head(6)
lbound = []
ubound = []
columns = df_features.columns
for col in columns:
    
    #degree of freedom = 0
    ci = bs.bootstrap(df_features[col].values,stat_func=bs_stats.std,alpha = 0.1)
    lbound.append(ci.lower_bound)
    ubound.append(ci.upper_bound)
    
pd.DataFrame({"Time-domain Features":columns, "Standard Deviation":df_features.std(),"Lower Bound(CI)":lbound,"Upper Bound(CI)":ubound})

,Time-domain Features,Standard Deviation,Lower Bound(CI),Upper Bound(CI)
Q1_1,Q1_1,6.153590,5.636627,6.697382
Q3_1,Q3_1,5.145600,4.410373,5.936963
max_1,max_1,4.394362,3.473805,5.404558
mean_1,mean_1,5.336157,4.762368,5.925217
median_1,median_1,5.440054,4.849297,6.061496
min_1,min_1,9.569975,8.290275,10.822066
std_1,std_1,1.772415,1.584877,1.959926
Q1_2,Q1_2,0.946386,0.853096,1.059230
Q3_2,Q3_2,2.125266,1.944100,2.338687
max_2,max_2,5.062729,4.694877,5.468656


iv. Use your judgement to select the three most important time-domain features.

### I select:

1. Mean: As data distribution is continuous.
2. Median: It is a robust measure least affected by outliers.
3. Standard deviation: It measures the dispersion in data.

### 2. ISLR 3.7.4

I collect a set of data (n = 100 observations) containing a single
predictor and a quantitative response. I then fit a linear regression
model to the data, as well as a separate cubic regression, i.e. Y =
β0 + β1X + β2X2 + β3X3 + ϵ.

#### (a) Suppose that the true relationship between X and Y is linear, i.e. Y = β0 + β1X + ϵ. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

Answer: I expect the RSS for the cubic regression to be lower than that of linear regression because a flexible model will fit more closely variance in the training data despite that the true relationship between X and Y is linear.

#### b) Answer (a) using test rather than training RSS.

Answer: For test RSS, a linear model will perform better as the assumption of high bias is correct. A cubic model on the other hand overfits the training data. The fit of this model will be affected by the variance in the training data. 

#### (c) Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

Answer: The training RSS will reduce with increase in "flexibility" regardless of what the true underlying model is. Training RSS for cubic regression will be less than that of linear regression as it will fit more closely variance in the training data.

#### (d) Answer (c) using test rather than training RSS.

Answer: There is not enough information to tell. 

- If the **true underlying model is simple**, we expect the **RSS for linear regression model to be lower** than that of cubic regression because our assumption of low bias was true. The cubic regression model will overfit the training data.

- If the **true underlying model is highly non-linear**, we expect the **RSS for cubic regression(flexible model)** to be lower than that of linear regression because the true model has high variance. Linear regression will probably underfit the training data.